Librerías

In [ ]:
import pandas as pd
import snowflake.connector
import numpy as np
from datetime import datetime, timedelta

from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaFileUpload

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import io
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders

Snowflake connector

In [ ]:
conn = snowflake.connector.connect(user='esteban.correa@RAPPI.COM', 
                                   authenticator='externalbrowser', 
                                   account='hg51401', 
                                   warehouse="RP_PERSONALUSER_WH",
                                   database="FIVETRAN")

Función para SF

##### SWA, AVL, conversión, ingresos y service level Rappi

In [ ]:
swa = """with 

avl_closing as (
select final.warehouseid as warehouse_id, final.sync_wh_id, d.external_id as dependencia, final.warehouse_name, final.storereferenceid as storereference_id, final.sync_product_id,
ep.plu_padre, ep.plu_hijo, ep."FACTOR", e.ean, final.sales_ayer, final.full_sales_28, final.available_hours, final.total_hours, final.full_available_hours, final.full_total_hours, final.stock_units as stock_rappi
from (
select a.warehouseid, w.warehouse_id as sync_wh_id, a.warehousename as warehouse_name, a.storereferenceid, p.product_id as sync_product_id, a.sales_units as sales_ayer, a.available_hours, a.total_hours, a.full_available_hours, a.full_total_hours, a.full_sales_28,a.stock_units --a.wl_type, 
from RP_SILVER_DB_PROD.TURBO_CORE.global_closing_inventory_current as a
left join fivetran.co_amysql_turbo_emergency_order_turbo_sync.warehouse_integration as w
on w.external_id = a.warehouseid
    left join fivetran.co_amysql_turbo_emergency_order_turbo_sync.product_integration as p
on p.external_id = a.storereferenceid
where a.country_code = 'CO' 
and a.main_date = current_date - interval '1 day'
and p._fivetran_deleted = 'FALSE'
and w.integration_name = 'vivo'
and p.integration_name = 'vivo') as final
left join fivetran.co_amysql_turbo_emergency_order_turbo_sync.warehouse_integration as d
on d.warehouse_id = final.sync_wh_id
left join fivetran.co_amysql_turbo_emergency_order_turbo_sync.product as e
on e.id = final.sync_product_id
left join FIVETRAN.CPGS_TURBO_DS_PUBLIC.CO_FILL_RATE_EXITO_PRODUCTO as ep
on ep.vivo_id = final.storereferenceid
where d.integration_name = 'exito'
and ean not in ('1009FLV','1099FLV','10CRAKT23','1128FLV','1142FLV','1150FLV','1154FLV','1164FLV','1207FLV','1218FLV','1244FLV','1257FLV','1288FLV','1293FLV','1313FLV','1375FLV','1418FLV','1432FLV','1461FLV',
'1581FLV','1943FLV','1948FLV','24ORCHB23','29ORALM23','LLB1904B','NOEAN1835901','NOEAN1835902','NOEAN1835903','NOEAN1835904','NOEAN1835905', '7,80E+12')),

closing_2 as (
select a.*, b.vertical, b.atribute, b.product_name, b.macrocategory_name, b.category_name, b.subcategory_name, b.bucket
from avl_closing as a
left join fivetran.cpgs_turbo_ds_public.global_product_supply as b
on a.warehouse_id = b.warehouseid and a.storereference_id = b.storereferenceid
having b.wl_type in ('1 Ideal','3 Substitute') 
and b.active in ('Active' , 'New', 'Inactive')
--and atribute = 'No R2E' 
and product_name != 'Out of catalogo' 
and b.vertical = 'Turbo'
and b.country_code = 'CO'),

wl as (
select city, warehouseid, storereferenceid, scope, wl_type
from fivetran.cpgs_turbo_ds_public.global_wishlist_with_pareto_new
where country_code = 'CO'),

closing_3 as (
select *, case when full_sales_28 < 0 then 0 else full_sales_28 end as vt, w.city as city_name, w.scope as scope_wh, w.wl_type as is_wl
from closing_2 as c
left join wl as w
on w.warehouseid = c.warehouse_id and w.storereferenceid = c.storereference_id),

recepcion as (
select cast(last_value(k.created_at) over (partition by l.warehouse_id, l.product_id order by k.created_at) as date) as reception_date,
l.warehouse_id, l.product_id, k.lot_stock
from fivetran.co_amysql_turbo_emergency_order_turbo_inventory_savvy_ms.kardex as k
left join fivetran.co_amysql_turbo_emergency_order_turbo_inventory_savvy_ms.lot as l 
on k.lot_id = l.id
left join fivetran.co_amysql_turbo_emergency_order_turbo_inventory_savvy_ms.kardex_type as kt
on kt.id = k.kardex_type_id 
where k.kardex_type_id in (1,19)),

transformation as (
select convert_timezone('America/Bogota', transformation_date) as transformation_date, warehouse_id, product_id, sum(transfor_units) as transfor_units
from (
select last_value(k.created_at) over (partition by l.warehouse_id, l.product_id order by k.created_at) as transformation_date,
l.warehouse_id, l.product_id, k.lot_stock as transfor_units
from fivetran.co_amysql_turbo_emergency_order_turbo_inventory_savvy_ms.kardex as k
left join fivetran.co_amysql_turbo_emergency_order_turbo_inventory_savvy_ms.lot as l 
on k.lot_id = l.id
left join fivetran.co_amysql_turbo_emergency_order_turbo_inventory_savvy_ms.kardex_type as kt
on kt.id = k.kardex_type_id 
where k.kardex_type_id in (5)
and k.detail = 'transformation_in_unit')
group by transformation_date, warehouse_id, product_id),

ns as (
select q1.WAREHOUSE_ID_NEW, q1.product_id, q1.planeado as planned_3, q1.entregado as delivered_3, 
(delivered_3 / nullif(planned_3,0)) as service_level_3, q2.planeado_2 as planned_lw, q2.entregado_2 as delivered_lw, 
(delivered_lw / nullif(planned_lw,0)) as service_level_lw
from (
select WAREHOUSE_ID_NEW, product_id, sum(planned) as planeado, sum(delivered) as entregado
from fivetran.cpgs_turbo_ds_public.global_po_main_current
where commited_date >= current_date() - interval '3 day'
and country_code = 'CO'
and status <> 'CANCELED'
group by WAREHOUSE_ID_NEW, product_id) as q1
inner join (
select WAREHOUSE_ID_NEW, product_id, sum(planned) as planeado_2, sum(delivered) as entregado_2
from fivetran.cpgs_turbo_ds_public.global_po_main_current
where commited_date between date_trunc('w', current_date - interval '7 days') and date_trunc('w', current_date) - interval '1 days'
and country_code = 'CO'
and status <> 'CANCELED'
group by WAREHOUSE_ID_NEW, product_id) as q2
on q1.WAREHOUSE_ID_NEW = q2.WAREHOUSE_ID_NEW and q1.product_id = q2.product_id),

top_80 as (
select storereferenceid, wl_name, cat as performance_catman
from fivetran.cpgs_turbo_ds_public.global_top_80
where country_code = 'CO'
and cat in ('4.-DEAD', '3.-UGLY_4')
and first_sale>60),

sacar_wl as (
select warehouseid, storereferenceid, keep_in_wl
from fivetran.cpgs_turbo_ds_public.co_portfolio_out_review 
where product_new = 'FALSE'),

merma as (
select lw.warehouseid as sync_wh_id, round(lw.storereferenceid,0) as sync_product_id,
shrinkage_usd_cw, shrinkage_units_cw,
round(sum(case when lw.type = 'KNOWN' then lw.usd_amount else 0 end),2) as shrinkage_usd_lw,
round(sum(case when lw.type = 'KNOWN' then lw.units else 0 end),2) as shrinkage_units_lw
from fivetran.cpgs_turbo_ds_public.global_finance_shrinkage as lw
left join (
select warehouseid as sync_wh_id, round(storereferenceid,0) as sync_product_id, 
round(sum(case when type = 'KNOWN' then usd_amount else 0 end),2) as shrinkage_usd_cw,
round(sum(case when type = 'KNOWN' then units else 0 end),2) as shrinkage_units_cw
from fivetran.cpgs_turbo_ds_public.global_finance_shrinkage
where country_code = 'CO'
and main_date >= date_trunc('w',current_date)
group by sync_wh_id, sync_product_id
) as cw
on lw.warehouseid = cw.sync_wh_id and round(lw.storereferenceid,0) = cw.sync_product_id
where lw.country_code = 'CO'
and lw.main_date between date_trunc('w',current_date) - interval '7 day' and date_trunc('w',current_date) - interval '1 day'
group by lw.warehouseid, lw.storereferenceid, shrinkage_usd_cw, shrinkage_units_cw),

forecast as (
select warehouse_id as sync_wh_id, product_id as sync_product_id, sum(forecast) as forecast_lw, sum(sales_units) as sales_lw
from RP_SILVER_DB_PROD.TURBO_CORE.GLOBAL_FORECAST_MAIN
where country = 'CO'
and date between date_trunc('week', current_date - interval '7 day') and date_trunc('week', current_date) - interval '1 day'
group by sync_wh_id, sync_product_id),

sales28 as (
select warehouse_id as sync_wh_id, product_id as sync_product_id, sum(forecast) as forecast_lw, sum(sales_units) as sales_28
from RP_SILVER_DB_PROD.TURBO_CORE.GLOBAL_FORECAST_MAIN
where country = 'CO'
and date between date_trunc('week', current_date - interval '28 day') and date_trunc('week', current_date) - interval '1 day'
group by sync_wh_id, sync_product_id)


select c.city_name, c.warehouse_id, c.sync_wh_id, c.dependencia, c.warehouse_name, c.storereference_id, c.sync_product_id, c.plu_padre, c.plu_hijo, c."FACTOR", 
cast(c.ean as numeric) as ean, c.is_wl, c.scope_wh, c.product_name, c.macrocategory_name, c.category_name, c.subcategory_name, c.bucket, c.stock_rappi,
c.sales_ayer, f.forecast_lw, f.sales_lw, round(div0(sum(c.full_available_hours),sum(c.full_total_hours))*100,2) as avl_nacional, r.reception_date, 
to_varchar(t.transformation_date, 'YYYY-MM-DD HH24:MI:SS') as transformation_date, t.transfor_units, zeroifnull(n.planned_3) as planned_3, 
zeroifnull(n.delivered_3) as delivered_3, zeroifnull(n.service_level_3) as service_level_3, zeroifnull(n.planned_lw) as planned_lw, 
zeroifnull(n.delivered_lw) as delivered_lw, zeroifnull(service_level_lw) as service_level_lw, p.performance_catman, 
case when s.keep_in_wl = 'Se recomienda sacar de la WL' then 1 else 0 end as remove_wl, zeroifnull(shrinkage_usd_cw) as shrinkage_usd_cw, 
zeroifnull(shrinkage_units_cw) as shrinkage_units_cw, zeroifnull(shrinkage_usd_lw) as shrinkage_usd_lw, 
zeroifnull(shrinkage_units_lw) as shrinkage_units_lw
from closing_3 as c
left join recepcion as r
on r.warehouse_id = c.sync_wh_id and r.product_id = c.sync_product_id
left join transformation as t
on t.warehouse_id = c.sync_wh_id and t.product_id = c.sync_product_id
left join ns as n
on n.WAREHOUSE_ID_NEW = c.warehouse_id and n.product_id = c.storereference_id
left join top_80 as p
on p.wl_name = c.scope_wh and p.storereferenceid = c.storereference_id
left join sacar_wl as s 
on s.warehouseid = c.warehouse_id and s.storereferenceid = c.storereference_id
left join merma as m
on m.sync_wh_id = c.sync_wh_id and m.sync_product_id = c.sync_product_id
left join forecast as f
on f.sync_wh_id = c.sync_wh_id and f.sync_product_id = c.sync_product_id
left join sales28 as s28
on s28.sync_wh_id = c.sync_wh_id and s28.sync_product_id = c.sync_product_id
group by c.city_name, c.warehouse_id, c.sync_wh_id, c.dependencia, c.warehouse_name, c.storereference_id, c.sync_product_id, c.plu_padre, c.plu_hijo, c."FACTOR", 
c.ean, c.is_wl, c.scope_wh, c.product_name, c.macrocategory_name, c.category_name, c.subcategory_name, c.bucket, c.stock_rappi, c.sales_ayer, f.forecast_lw,
f.sales_lw, s28.sales_28,  c.vt, c.full_available_hours, c.full_total_hours, r.reception_date, t.transformation_date, 
t.transfor_units, n.planned_3, n.delivered_3, service_level_3, n.planned_lw, n.delivered_lw, service_level_lw, p.performance_catman, s.keep_in_wl, 
shrinkage_usd_cw, shrinkage_units_cw, shrinkage_usd_lw, shrinkage_units_lw 
order by f.forecast_lw desc;
"""

In [ ]:
df_swa = pd.read_sql(swa,conn)

In [ ]:
def convertir_nombres_a_minusculas(df):
    df.columns = [col.lower() for col in df.columns]
    return df

df_swa = convertir_nombres_a_minusculas(df_swa)

df_swa.head()

df_swa.loc[df_swa['plu_padre'] == '255010'].head()

##### Archivo éxito

In [ ]:
exito = pd.read_excel(r"C:\Users\juane\Downloads\base_exito.xlsx")
exito.loc[exito['Plu PluCD'] == 255010].head()

In [ ]:
columnas_no_eliminar = ['Dependencia DependenciaCD', 'Plu PluCD', 'Proveedor NombreProveedor', 'Proveedor Nit', 
                        'Descripcionestadoplu ID', 'Cediatiende ID', 'Inventario', 'PedidoPendiente', 'StockDePresentacion', 
                        'InventarioMaximo','Descripcionestadoplu ID', 'Umd']

exito.drop(columns=[col for col in exito if col not in columnas_no_eliminar], inplace=True)
exito.head(3)
exito.loc[exito['Plu PluCD'] == 255010].head()

In [ ]:
exito.rename(columns={'Dependencia DependenciaCD':'dependencia', 'Plu PluCD':'plu_padre', 
                      'Proveedor NombreProveedor':'supplier_name', 'Proveedor Nit':'nit_supplier',
                      'Descripcionestadoplu ID':'plu_status', 'Cediatiende ID':'cedi', 'Inventario':'stock_exito', 
                      'PedidoPendiente':'pp', 'StockDePresentacion':'sp', 'InventarioMaximo':'max_inventory', 
                      'Umd':'umd'}, inplace=True)
##exito.head(2)
exito.loc[exito['plu_padre'] == 255010].head()

##### Unión información

In [ ]:
# Limpiar 'plu_padre' en ambos DataFrames y convertir a número
df_swa['plu_padre'] = df_swa['plu_padre'].astype(str).str.replace('.0', '', regex=False)
df_swa['plu_padre'] = pd.to_numeric(df_swa['plu_padre'], errors='coerce').fillna(0).astype(int)

exito['plu_padre'] = exito['plu_padre'].astype(str).str.replace('.0', '', regex=False)
exito['plu_padre'] = pd.to_numeric(exito['plu_padre'], errors='coerce').fillna(0).astype(int)

# Limpiar 'dependencia' en ambos DataFrames y convertir a número
df_swa['dependencia'] = df_swa['dependencia'].astype(str).str.replace('.0', '', regex=False)
df_swa['dependencia'] = pd.to_numeric(df_swa['dependencia'], errors='coerce').fillna(0).astype(int)

exito['dependencia'] = exito['dependencia'].astype(str).str.replace('.0', '', regex=False)
exito['dependencia'] = pd.to_numeric(exito['dependencia'], errors='coerce').fillna(0).astype(int)

#print(df_swa[['dependencia', 'plu_padre']].head())
#print(exito[['dependencia', 'plu_padre']].head())
exito.loc[exito['plu_padre'] == 255010].head()

In [ ]:
# Convertir 'plu_padre' y 'dependencia' en números en ambos DataFrames
df_swa['plu_padre'] = pd.to_numeric(df_swa['plu_padre'], errors='coerce').fillna(0).astype(int)
exito['plu_padre'] = pd.to_numeric(exito['plu_padre'], errors='coerce').fillna(0).astype(int)

df_swa['dependencia'] = pd.to_numeric(df_swa['dependencia'], errors='coerce').fillna(0).astype(int)
exito['dependencia'] = pd.to_numeric(exito['dependencia'], errors='coerce').fillna(0).astype(int)

# Realizar el merge asegurando que ambas columnas son del mismo tipo
swa_exito = pd.merge(left=df_swa, right=exito, how='left', on=['dependencia', 'plu_padre'])

# Mostrar las primeras 10 filas
swa_exito.head(10)


In [ ]:
swa_exito = swa_exito.loc[:,['city_name', 'warehouse_id', 'sync_wh_id', 'dependencia', 'warehouse_name', 'storereference_id', 
                  'sync_product_id', 'plu_padre', 'plu_hijo', 'factor', 'ean', 'is_wl', 'scope_wh', 'product_name',
                  'macrocategory_name', 'category_name', 'subcategory_name', 'bucket', 'stock_rappi', 'stock_exito', 
                  'forecast_lw', 'sales_lw','sales_ayer', 'avl_nacional',
                  'reception_date', 'transfor_units', 'planned_3', 'delivered_3', 'service_level_3', 
                  'planned_lw', 'delivered_lw',
                  'shrinkage_usd_cw', 'shrinkage_units_cw', 'shrinkage_usd_lw', 'shrinkage_units_lw', 'supplier_name', 
                  'plu_status', 'cedi', 'pp', 'sp', 'max_inventory', 'umd']]



In [ ]:
# Verificar la forma del DataFrame resultante antes de eliminar duplicados
print("Dimensiones del DataFrame antes de eliminar duplicados:", swa_exito.shape)

# Convertir las columnas a los tipos de datos deseados
swa_exito_unique = swa_exito.astype({'dependencia':'float','avl_nacional':'float',
                                      'planned_3':'int','delivered_3':'int','service_level_3':'float',
                                      'planned_lw':'int','delivered_lw':'int'})

# Eliminar duplicados basados en 'warehouse_id' y 'sync_wh_id'
swa_exito_unique = swa_exito.drop_duplicates(subset=['warehouse_id', 'sync_product_id'])

# Verificar la forma del DataFrame resultante después de eliminar duplicados
print("Dimensiones del DataFrame después de eliminar duplicados:", swa_exito_unique.shape)

# Mostrar los primeros 8 registros del DataFrame resultante
print(swa_exito_unique.head(8))


In [ ]:
hoy = datetime.today().strftime('%Y-%m-%d')

swa_exito_unique.to_excel(f'Control_tower_supply_{hoy}.xlsx',index=False)

In [ ]:
# Replace with the path to your service account JSON file
creds = Credentials.from_service_account_file('optimal-torch-439314-q9-2c512ed2a125.json')

# Replace with the name of the folder where you want to upload the file
folder_name = 'ControlTower'

# Replace with the path to the Excel file you want to upload
file_path = f'Control_tower_supply_{hoy}.xlsx'

# Create a Drive API client
drive_service = build('drive', 'v3', credentials=creds)

# Find the ID of the target folder
query = "mimeType='application/vnd.google-apps.folder' and trashed=false and name='{}'".format(folder_name)
response = drive_service.files().list(q=query, fields='files(id)').execute()
folder_id = response.get('files', [])[0].get('id')

# Upload the Excel file to Drive
file_metadata = {'name': f"Control_tower_supply_{hoy}.xlsx", 'parents': [folder_id]}
media = MediaFileUpload(file_path, resumable=True)
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

# Give people edit access to the file
file_id = file.get('id')


new_permissions = [
    {'type': 'user', 'role': 'writer', 'emailAddress': 'johan.mayorga@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'marcel.kempe@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'julian.ariza@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'roland.rojas@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'antonio.fernandez@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'santiago.lozano@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'mariangela.uribe@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'santiago.martin@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'Helena.kuntz@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'juliana.zarate@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'andrea.cabrales@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'jose.russi@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'esteban.correa@rappi.com'},
    {'type': 'user', 'role': 'writer', 'emailAddress': 'maria.mosquera@rappi.com'}
]

for permission in new_permissions:
    drive_service.permissions().create(
        fileId=file_id,
        body={
            'kind': 'drive#permission',
            'type': 'user',
            'role': permission['role'],
            'emailAddress': permission['emailAddress']
        },
        sendNotificationEmail=True  # optional
    ).execute()

# Print the link to the file
link = "https://drive.google.com/file/d/{}/view?usp=sharing".format(file_id)
print("File uploaded to Drive and shared with the specified email addresses:")
print(link)

In [ ]:
# Configura las credenciales de tu cuenta de correo electrónico
sender_email = 'esteban.correa@rappi.com'
sender_password = 'yexw ulvz yakj ggpc'
remitente = """Rappi Turbo Supply"""


recipient_emails = ['juliana.zarate@rappi.com','naidys.yepes@rappi.com','santiago.martin@rappi.com', 
                    'mariangela.uribe@rappi.com','marcel.kempe@rappi.com',
                    'johan.mayorga@rappi.com','julian.ariza@rappi.com','antonio.fernandez@rappi.com',
                    'santiago.lozano@rappi.com', 'roland.rojas@rappi.com', 'Helena.kuntz@rappi.com','andrea.cabrales@rappi.com', 'jose.russi@rappi.com','esteban.correa@rappi.com','maria.mosquera@rappi.com']

# Construye el mensaje de correo electrónico en formato HTML
subject = f'Control_tower_supply_{hoy}.xlsx'
body = f"""Hola,<br><br>Adjunto encontrarán el <a href="{link}">INFORME</a> de agotados, SWA y alertas.<br><br>¡Gracias!"""

# Envía el correo electrónico a cada destinatario en la lista
for recipient_email in recipient_emails:
    message = MIMEMultipart()
    message['From'] = remitente
    message['To'] = recipient_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'html'))

    # Establece la conexión con el servidor SMTP de Gmail
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)

        # Envía el mensaje de correo electrónico
        server.sendmail(sender_email, recipient_email, message.as_string())
        print(f'Correo electrónico enviado con éxito a {recipient_email}')
    except Exception as e:
        print(f'Error al enviar el correo electrónico a {recipient_email}: {str(e)}')
    finally:
        # Cierra la conexión con el servidor SMTP
        server.quit()